In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

# path to json file that stores MFCCs and genre labels for each processed segment
DATA_PATH = "mfcc_data_20s.json"

def load_data(data_path):
    """Loads training dataset from json file.

        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    X = np.array(data["mfccs"])
    y = np.array(data["labels"])
    genres = data["genres"]

    print("Data succesfully loaded!")

    return  X, y, genres

In [2]:
# load data
X, y, genres = load_data(DATA_PATH)

Data succesfully loaded!


In [3]:
# create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [4]:
# build network topology
model = keras.Sequential([

    # input layer
    keras.layers.Flatten(input_shape=(X.shape[1], X.shape[2])),

    # 1st dense layer
    keras.layers.Dense(512, activation='relu'),

    # 2nd dense layer
    keras.layers.Dense(256, activation='relu'),

    # 3rd dense layer
    keras.layers.Dense(64, activation='relu'),

    # output layer
    keras.layers.Dense(len(genres), activation='softmax')
])

# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 17240)             0         
                                                                 
 dense (Dense)               (None, 512)               8827392   
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 64)                16448     
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 8,975,818
Trainable params: 8,975,818
Non-trainable params: 0
_________________________________________________________________


In [5]:
# train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=50)

Epoch 1/50
22/22 [==============================] - 3s 110ms/step - loss: 26.2031 - accuracy: 0.2346 - val_loss: 17.9074 - val_accuracy: 0.2800
Epoch 2/50
22/22 [==============================] - 2s 101ms/step - loss: 11.8178 - accuracy: 0.3720 - val_loss: 9.2256 - val_accuracy: 0.3367
Epoch 3/50
22/22 [==============================] - 2s 99ms/step - loss: 4.2038 - accuracy: 0.4721 - val_loss: 5.7873 - val_accuracy: 0.3833
Epoch 4/50
22/22 [==============================] - 2s 99ms/step - loss: 2.0659 - accuracy: 0.5351 - val_loss: 3.4551 - val_accuracy: 0.3567
Epoch 5/50
22/22 [==============================] - 2s 104ms/step - loss: 1.3754 - accuracy: 0.5665 - val_loss: 3.2638 - val_accuracy: 0.3400
Epoch 6/50
22/22 [==============================] - 2s 96ms/step - loss: 1.0956 - accuracy: 0.6309 - val_loss: 3.5978 - val_accuracy: 0.3700
Epoch 7/50
22/22 [==============================] - 2s 101ms/step - loss: 0.8509 - accuracy: 0.6981 - val_loss: 3.6498 - val_accuracy: 0.3400
Epoch 

In [6]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=64)
print("test loss, test acc:", results)

Evaluate on test data
5/5 [==============================] - 0s 12ms/step - loss: 5.8807 - accuracy: 0.4867
test loss, test acc: [5.880698204040527, 0.4866666793823242]


In [7]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for tests")
predictions = model.predict(X_test)
correct = 0
for i in range(predictions.shape[0]):
    max_confidence = max(predictions[i])
    pred = np.argmax(predictions[i])
    actual = y_test[i]
    if pred == actual:
        correct += 1
    print('Prediction:', genres[pred], '\tActual:', genres[actual], '\nConfidence:', max(predictions[i]),'\n')

Generate predictions for tests
10/10 [==============================] - 0s 7ms/step
Prediction: rock 	Actual: disco 
Confidence: 0.9990683 

Prediction: blues 	Actual: country 
Confidence: 0.8473773 

Prediction: jazz 	Actual: jazz 
Confidence: 0.99993503 

Prediction: rock 	Actual: rock 
Confidence: 0.8048592 

Prediction: reggae 	Actual: reggae 
Confidence: 0.9739715 

Prediction: metal 	Actual: hiphop 
Confidence: 0.73423684 

Prediction: country 	Actual: rock 
Confidence: 0.97374356 

Prediction: classical 	Actual: classical 
Confidence: 0.99998736 

Prediction: jazz 	Actual: pop 
Confidence: 0.99876934 

Prediction: classical 	Actual: classical 
Confidence: 1.0 

Prediction: pop 	Actual: pop 
Confidence: 0.99999523 

Prediction: pop 	Actual: pop 
Confidence: 1.0 

Prediction: jazz 	Actual: reggae 
Confidence: 0.99831796 

Prediction: classical 	Actual: classical 
Confidence: 1.0 

Prediction: blues 	Actual: country 
Confidence: 0.78811103 

Prediction: disco 	Actual: disco 
Confid

In [8]:
print(correct / predictions.shape[0] * 100,'% correctly predicted')

48.66666666666667 % correctly predicted
